Mounting the Drive (Optional Step)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Clone the Dataset into your environment

In [ ]:
!kaggle datasets download -d pratik2901/multiclass-weather-dataset

Dataset URL: https://www.kaggle.com/datasets/pratik2901/multiclass-weather-dataset
License(s): Attribution 4.0 International (CC BY 4.0)
 89% 81.0M/91.4M [00:01<00:00, 70.9MB/s]
100% 91.4M/91.4M [00:01<00:00, 70.6MB/s]


Extract the Cloned Zip files

In [ ]:
import zipfile

In [ ]:
with zipfile.ZipFile('/content/multiclass-weather-dataset.zip', 'r') as zip_ref:
  zip_ref.extractall('/content')

Using the img2vec -pytorch function. This will allow us to extarct the feature in the vectorized form by just passing the RGB Image

In [ ]:
!pip install img2vec-pytorch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 

Using other necessary libraries:


1.   Pillow: For Image Manipulation
2.   Scikit-Learn: for Machine Learning Algorithms



In [ ]:
!pip install pillow
!pip install scikit-learn

Importing the libraries, we need:

In [ ]:
import os
import pickle

from img2vec_pytorch import Img2Vec
from PIL import Image
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

This will extact the dataset(replace the name) also make sure to make the train and cal directories into the dataset before runnning this cell.

In [ ]:


# Initialize the Img2Vec model
img2vec = Img2Vec()

data_dir = '/content/drive/MyDrive/weather_dataset'
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')

data = {}
for j, dir_ in enumerate([train_dir, val_dir]):
    features = []
    labels = []
    for category in os.listdir(dir_):
        for img_path in os.listdir(os.path.join(dir_, category)):
            img_path_ = os.path.join(dir_, category, img_path)
            img = Image.open(img_path_)

            # Ensure the image is in RGB format
            img = img.convert('RGB')

            img_features = img2vec.get_vec(img)

            features.append(img_features)
            labels.append(category)

    data[['training_data', 'validation_data'][j]] = features
    data[['training_labels', 'validation_labels'][j]] = labels


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 88.0MB/s]


Train the Model

In [ ]:
# train model

model = RandomForestClassifier(random_state=0)
model.fit(data['training_data'], data['training_labels'])



RandomForestClassifier(random_state=0)

Check the Accuracy

In [ ]:
# test performance
y_pred = model.predict(data['validation_data'])
score = accuracy_score(y_pred, data['validation_labels'])

print(score)



1.0


Save the Model

In [ ]:
# save the model
with open('./model.p', 'wb') as f:
    pickle.dump(model, f)
    f.close()

Take the Predictions from the Model to some random Images

In [ ]:
import pickle

from img2vec_pytorch import Img2Vec
from PIL import Image


with open('/content/drive/MyDrive/weather_dataset/model.p', 'rb') as f:
    model = pickle.load(f)

img2vec = Img2Vec()

image_path = '/content/drive/MyDrive/weather_dataset/tt.jpg'

img = Image.open(image_path)

features = img2vec.get_vec(img)

pred = model.predict([features])
print(score)
print(pred)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


1.0
['Rain']
